# STEP 1 — Load CSV datafile (FAST I/O version)
#
# This cell:
#  1) left memory_map at its (faster) default True
#  2) supply a date_format, which lets pandas use a vectorized, fast path.

In [10]:
#import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path

In [11]:
# Define the path to the CSV file and output file
IN = Path("nytaxi2022.csv")
OUT = Path("nytaxi2022_cleaned.npz")

In [16]:
# Columns to be used
use_cols = [
    "tpep_pickup_datetime","tpep_dropoff_datetime","passenger_count","trip_distance",
    "RatecodeID","PULocationID","DOLocationID","payment_type","extra","total_amount"
]


# Exact datetime format (your file uses this)
DT_FMT = "%m/%d/%Y %I:%M:%S %p"

# Small, memory-friendly dtypes (nullable ints for possible NAs)
# If extension Int* feels slow on your box, switch Int* -> float32 and cast later.
dtypes = {
    "passenger_count": "float32",
    "trip_distance":   "float32",
    "RatecodeID":      "Int8",
    "PULocationID":    "Int16",
    "DOLocationID":    "Int16",
    "payment_type":    "Int8",
    "extra":           "float32",
    "total_amount":    "float32",
}

print("Reading CSV fast-path…")
df = pd.read_csv(
    IN,
    usecols=use_cols,
    dtype=dtypes,
    parse_dates=["tpep_pickup_datetime","tpep_dropoff_datetime"],
    date_format=DT_FMT,      # <<< critical for speed
    engine="c",
    memory_map=True
)
print(df.shape)
print(df.head())

Reading CSV fast-path…
(39656098, 10)
  tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  trip_distance  \
0  2022-01-01 00:35:40   2022-01-01 00:53:29              2.0           3.80   
1  2022-01-01 00:33:43   2022-01-01 00:42:07              1.0           2.10   
2  2022-01-01 00:53:21   2022-01-01 01:02:19              1.0           0.97   
3  2022-01-01 00:25:21   2022-01-01 00:35:23              1.0           1.09   
4  2022-01-01 00:36:48   2022-01-01 01:14:20              1.0           4.30   

   RatecodeID  PULocationID  DOLocationID  payment_type  extra  total_amount  
0           1           142           236             1    3.0     21.950001  
1           1           236            42             1    0.5     13.300000  
2           1           166           166             1    0.5     10.560000  
3           1           114            68             2    0.5     11.800000  
4           1            68           163             1    0.5     30.299999  


In [17]:
# Keep a copy of the original dataframe for reference
df_original = df.copy()

# STEP 2 — Clean up the dataset
#
# This cell:
#  1) Drop rows with NA in critical columns
#  2) Filter out invalid trips (trip_duration > 600 mins, trip_distance > 200 miles)

In [18]:
# Basic filtering / feature engineering
# 1) Drop rows with NA in critical columns
df = df.dropna(subset=["tpep_pickup_datetime","tpep_dropoff_datetime","total_amount"])
print("After dropna:", df.shape)

After dropna: (39656098, 10)


In [19]:
# Define trip duration in minutes
df["trip_duration_min"] = (
    (df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]).dt.total_seconds() / 60.0
).astype("float32")

# 2) Filter out invalid trips
df = df[
    (df["trip_duration_min"] > 0) & (df["trip_duration_min"] < 600) &  # max 10 hours
    (df["trip_distance"] > 0) & (df["trip_distance"] < 200)]  # max 200 miles

print("After filtering:", df.shape)


After filtering: (39021102, 11)


In [20]:
# Time features
df["hour"] = df["tpep_pickup_datetime"].dt.hour.astype("int16")
df["dow"]  = df["tpep_pickup_datetime"].dt.dayofweek.astype("int16")
df["is_weekend"] = (df["dow"]>=5).astype("int8")

# Categorical (small-cardinality) one-hots
for col, cats in [("payment_type", sorted(df["payment_type"].dropna().unique())),
                  ("RatecodeID",  sorted(df["RatecodeID"].dropna().unique()))]:
    dummies = pd.get_dummies(df[col].fillna(-1).astype(int), prefix=col)
    df = pd.concat([df, dummies], axis=1)

# Keep numeric/simple encodings
num_cols = [
    "passenger_count","trip_distance","extra","trip_duration_min","hour","dow","is_weekend",
    "PULocationID","DOLocationID"
]
one_hot_cols = [c for c in df.columns if c.startswith("payment_type_") or c.startswith("RatecodeID_")]
X_cols = num_cols + one_hot_cols
y_col = "total_amount"

df = df.dropna(subset=X_cols+[y_col])
X = df[X_cols].to_numpy(dtype=np.float32)
y = df[y_col].to_numpy(dtype=np.float32)

In [ ]:
# Scale numerics by train stats later; split first (stratification not needed)
N = len(X)
RNG = np.random.default_rng(42)
idx = RNG.permutation(N)
cut = int(0.7*N)
train_idx, test_idx = idx[:cut], idx[cut:]

X_train, X_test = X[train_idx], X[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Standardize numerics using train stats
num_idx = [X_cols.index(c) for c in num_cols]
mu = X_train[:, num_idx].mean(axis=0)
sd = X_train[:, num_idx].std(axis=0) + 1e-8

In [22]:
def apply_scale(A):
    A = A.copy()
    A[:, num_idx] = (A[:, num_idx]-mu)/sd
    # scale location IDs (already inside num_cols) into [0,1] column-wise
    pu_i, do_i = X_cols.index("PULocationID"), X_cols.index("DOLocationID")
    for ii in [pu_i, do_i]:
        col = A[:, ii]
        mn, mx = col.min(), col.max()
        A[:, ii] = (col - mn) / (mx - mn + 1e-8)
    return A

X_train = apply_scale(X_train)
X_test  = apply_scale(X_test)

Path("data").mkdir(exist_ok=True)
np.savez(OUT,
         X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,
         X_cols=np.array(X_cols, dtype=object), mu=mu, sd=sd, num_idx=np.array(num_idx))
print(f"Saved {OUT}  | rows: train={len(X_train)}, test={len(X_test)}")

Saved nytaxi2022_cleaned.npz  | rows: train=26416958, test=11321554
